In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import glob, os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
def getPadding(data, max_length):
    return sequence.pad_sequences(data, padding='post', maxlen=max_length)

In [3]:
dir = './sequence/*'

In [4]:
# file read
all_data = []
for file in sorted(glob.glob(dir)):
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    all_data.append(datasets)

In [18]:
x_train, x_test = train_test_split(all_data, test_size = 0.2)

In [19]:
sequence_length_tr = []
sequence_length_te = []
x_train = sorted(x_train, key=len)[::-1]
x_test = sorted(x_test, key=len)[::-1]

for xr in x_train:
    sequence_length_tr.append(len(xr))
for xe in x_test:
    sequence_length_te.append(len(xe))

#print(sequence_length_tr, sequence_length_te)
#len(x_train), len(x_test)
len(sequence_length_tr), len(sequence_length_te)

(540, 135)

In [21]:
max_sequence_length = max(max(sequence_length_tr), max(sequence_length_te))
n_features = 3
batch_size = 1
epochs = 1000
tr_num = len(x_train)
seq_lengths = sequence_length_tr

In [22]:
x_train = getPadding(x_train, max_sequence_length)
x_test = getPadding(x_test, max_sequence_length)

In [23]:
batch_in = torch.zeros((len(x_train), max_sequence_length, 3))

In [24]:
for (idx, x) in enumerate(x_train):
    batch_in[idx] = torch.FloatTensor(x)

In [25]:
batch_in = Variable(batch_in)

In [26]:
pack = torch.nn.utils.rnn.pack_padded_sequence(batch_in, seq_lengths, batch_first=True)#, enforce_sorted=False)

In [27]:
hidden_size = 2
n_layers = 1

In [28]:
rnn = nn.RNN(3, hidden_size, n_layers, batch_first = True)

In [29]:
h0 = Variable(torch.randn(n_layers, 540, hidden_size))

In [30]:
out, _ = rnn(pack, h0)

In [31]:
unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(out)

In [32]:
print(unpacked.size())

torch.Size([41, 540, 2])


In [33]:
print(unpacked_len)

tensor([41, 41, 41, 41, 41, 41, 41, 41, 40, 40, 40, 40, 40, 40, 40, 40, 40, 39,
        39, 39, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38,
        38, 38, 38, 38, 38, 38, 36, 36, 36, 36, 36, 36, 36, 36, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 32, 32, 32, 32, 32, 32, 32, 32, 31, 31, 31, 31,
        31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31,
        31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30, 30, 30, 30, 30, 30, 30, 30,
        30, 30, 30, 30, 30, 30, 30, 30, 29, 29, 29, 29, 29, 29, 29, 29, 28, 28,
        28, 28, 28, 28, 28, 27, 27, 27, 27, 27, 27, 27, 27, 27, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26,
        26, 26, 26, 25, 25, 25, 25, 25, 

In [ ]:
print(unpacked[2, ...])